In [1]:
import torch
from torch.utils.data import Dataset

class StockSlidingWindowDataset(Dataset):
    def __init__(self, price_feats, sentiment_feats, targets, window=20):
        """
        price_feats: Tensor [T, P]
        sentiment_feats: Tensor [T, S]
        targets: Tensor [T]
        """
        self.price = price_feats
        self.sentiment = sentiment_feats
        self.targets = targets
        self.window = window

    def __len__(self):
        return len(self.targets) - self.window

    def __getitem__(self, idx):
        X_price = self.price[idx:idx+self.window]
        X_sent = self.sentiment[idx:idx+self.window]
        X = torch.cat([X_price, X_sent], dim=1)
        y = self.targets[idx + self.window]
        return X, y


In [2]:
import torch.nn as nn

class SentimentLSTM(nn.Module):
    def __init__(self, input_size, hidden_size=64, layers=2, dropout=0.3):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size, hidden_size,
            num_layers=layers,
            batch_first=True,
            dropout=dropout
        )
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = out[:, -1, :]
        out = self.dropout(out)
        return self.fc(out)


In [3]:
from torch.utils.data import DataLoader
import torch.optim as optim

def train_lstm(model, loader, epochs=20):
    loss_fn = nn.MSELoss()
    opt = optim.Adam(model.parameters(), lr=1e-3)

    for e in range(epochs):
        total = 0
        for X, y in loader:
            opt.zero_grad()
            pred = model(X).squeeze()
            loss = loss_fn(pred, y)
            loss.backward()
            opt.step()
            total += loss.item()
        print(f"Epoch {e+1}, Loss: {total/len(loader):.4f}")


In [15]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
finbert = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")
finbert.eval()

# --- Start of added code to define 'loader' ---
# Dummy data for demonstration. In a real scenario, this would come from actual data.
sequence_length = 100
price_features_dim = 3
sentiment_features_dim = 3

# Generate random dummy data for price, sentiment features and targets
dummy_price_feats = torch.randn(sequence_length, price_features_dim)
dummy_sentiment_feats = torch.randn(sequence_length, sentiment_features_dim)
dummy_targets = torch.randn(sequence_length)

# Instantiate the dataset
dataset = StockSlidingWindowDataset(
    price_feats=dummy_price_feats,
    sentiment_feats=dummy_sentiment_feats,
    targets=dummy_targets,
    window=20
)

# Instantiate the DataLoader
loader = DataLoader(dataset, batch_size=32, shuffle=True)
# --- End of added code ---

lstm = SentimentLSTM(input_size=price_features_dim + sentiment_features_dim)  # Adjust input_size to match dummy data
train_lstm(lstm, loader, epochs=20)

torch.save(lstm.state_dict(), "sentiment_lstm.pth")
print("LSTM model saved successfully.")

lstm = SentimentLSTM(input_size=price_features_dim + sentiment_features_dim)
lstm.load_state_dict(torch.load("sentiment_lstm.pth", map_location="cpu"))
lstm.eval()

print("LSTM model loaded successfully.")

Epoch 1, Loss: 1.0020
Epoch 2, Loss: 0.9018
Epoch 3, Loss: 0.9764
Epoch 4, Loss: 0.9255
Epoch 5, Loss: 1.0820
Epoch 6, Loss: 0.9808
Epoch 7, Loss: 0.8376
Epoch 8, Loss: 0.9367
Epoch 9, Loss: 1.0780
Epoch 10, Loss: 0.8731
Epoch 11, Loss: 0.9484
Epoch 12, Loss: 0.9787
Epoch 13, Loss: 0.9258
Epoch 14, Loss: 0.9161
Epoch 15, Loss: 0.9776
Epoch 16, Loss: 0.8313
Epoch 17, Loss: 0.9337
Epoch 18, Loss: 0.8569
Epoch 19, Loss: 0.8687
Epoch 20, Loss: 0.8300
LSTM model saved successfully.
LSTM model loaded successfully.


In [16]:
import yfinance as yf
import pandas as pd
import numpy as np

stocks = ["AAPL", "MSFT", "GOOGL", "NVDA", "TSLA"]

prices = yf.download(stocks, period="2d", interval="1d", auto_adjust=True)


[*********************100%***********************]  5 of 5 completed


In [20]:
returns = {}
for s in stocks:
    close_yesterday = prices[('Close', s)].iloc[0]
    close_today = prices[('Close', s)].iloc[-1]
    returns[s] = (close_today - close_yesterday) / close_yesterday

In [22]:
news_stub = {
    "AAPL": "Strong earnings and services growth",
    "MSFT": "Cloud and AI momentum continues",
    "GOOGL": "Advertising stable, AI investments ongoing",
    "NVDA": "AI demand remains very strong",
    "TSLA": "Analyst upgrade but delivery miss"
}

def get_sentiment_score(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = finbert(**inputs)

    # Apply softmax to get probabilities
    probabilities = torch.softmax(outputs.logits, dim=1)


    negative_score = probabilities[0][0].item()
    neutral_score = probabilities[0][1].item()
    positive_score = probabilities[0][2].item()


    sentiment_score = positive_score - negative_score
    return sentiment_score

def decision(ret, sent):
    if ret > 0 and sent > 0:
        return "BUY 📈"
    elif ret < 0 and sent < 0:
        return "SELL 📉"
    else:
        return "HOLD ⏸️"

report = []

for s in stocks:
    sent_score = get_sentiment_score(news_stub[s])
    report.append({
        "Stock": s,
        "Today Return (%)": round(returns[s]*100, 2),
        "Sentiment Score": round(sent_score, 3),
        "Recommendation": decision(returns[s], sent_score)
    })

df_report = pd.DataFrame(report)
df_report

,Stock,Today Return (%),Sentiment Score,Recommendation
0,AAPL,-0.12,-0.885,SELL 📉
1,MSFT,NaN,0.513,HOLD ⏸️
2,GOOGL,NaN,0.604,HOLD ⏸️
3,NVDA,NaN,-0.819,HOLD ⏸️
4,TSLA,0.18,0.757,BUY 📈
